In [2]:
import pandas as pd
import numpy as np
import os

C:\Users\3ynut\AppData\Local\Temp\ipykernel_10764\2619883542.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# โหลดข้อมูลจากไฟล์ Excel
df_main = pd.read_excel('.\\data\\ความชื้นสัมพัทธ์ 2013-2022.xlsx')
df_main.drop(['เฉลี่ย', 'week1', 'Unnamed: 35',
         'Unnamed: 36', 'Unnamed: 37'], axis=1, inplace=True)
df_main['เดือน/ปี'] = pd.to_datetime(df_main['เดือน/ปี'])
# แยกข้อมูลในคอลัมน์ 'เดือน/ปี' เพื่อสร้างคอลัมน์ 'Year' และ 'Month'
df_main['Year'] = df_main['เดือน/ปี'].dt.year
df_main['Month'] = df_main['เดือน/ปี'].dt.month
df_main.drop(['เดือน/ปี'], axis=1, inplace=True)
df_main.head()

,สถานี,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,Year,Month
0,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,82,80,82,78,76,78,80,78,77,...,72,70,70,70,71,75,87,84,2013,1
1,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,77,82,77,72,72,70,70,72,72,...,69,66,66,60,59,-,-,-,2013,2
2,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,61,66,64,73,73,67,65,69,57,...,54,56,54,52,51,54,56,60,2013,3
3,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,57,53,46,47,51,49,49,52,52,...,58,49,50,49,48,61,68,-,2013,4
4,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,64,66,76,78,74,75,82,72,68,...,78,79,73,66,66,63,64,67,2013,5


In [4]:
df = df_main.copy()

# สร้างคอลัมน์ใหม่ที่เก็บข้อมูลทั้งหมดในแต่ละแถว
df['value'] = df.iloc[:, 1:-2].apply(list, axis=1)

grouped_df = df.groupby(['สถานี', 'Year'])[
    'value'].apply(list).reset_index()

grouped_df['value'] = grouped_df['value'].apply(lambda x: np.concatenate(x))
grouped_df.head()

,สถานี,Year,value
0,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,"[82, 80, 82, 78, 76, 78, 80, 78, 77, 77, 77, 7..."
1,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2014,"[84, 84, 84, 82, 80, 81, 79, 78, 78, 79, 82, 8..."
2,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2015,"[78, 77, 78, 80, 80, 80, 79, 84, 92, 79, 78, 7..."
3,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2016,"[76, 77, 77, 76, 77, 76, 76, 76, 78, 78, 74, 7..."
4,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2017,"[81, 80, 86, 92, 88, 83, 81, 79, 80, 88, 89, 8..."


In [5]:
week1 = pd.DataFrame(grouped_df.loc[0]).transpose()
week1['value'] = week1['value'].apply(lambda x: x[:5])
numeric_values = list(map(int, week1['value'][0][0:5]))
week1['start_of_week'] = 0
week1['Humi'] = np.mean(numeric_values)
week1.drop(['value'], axis=1, inplace=True) if 'value' in week1.columns else None
week1

,สถานี,Year,start_of_week,Humi
0,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,0,79.6


In [6]:
grouped_df['value'][0][0:5] = '-'
grouped_df.head()

,สถานี,Year,value
0,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,"[-, -, -, -, -, 78, 80, 78, 77, 77, 77, 74, 77..."
1,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2014,"[84, 84, 84, 82, 80, 81, 79, 78, 78, 79, 82, 8..."
2,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2015,"[78, 77, 78, 80, 80, 80, 79, 84, 92, 79, 78, 7..."
3,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2016,"[76, 77, 77, 76, 77, 76, 76, 76, 78, 78, 74, 7..."
4,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2017,"[81, 80, 86, 92, 88, 83, 81, 79, 80, 88, 89, 8..."


In [7]:
# นำข้อมูลทั้งหมดมาหาค่าเฉลี่ยเป็นรายสัปดาห์ แล้วนำมารวมกันในคอลัมน์ 'weekly_average'
def calculate_weekly_average(values):
    # แปลงค่าในรายการเป็นจำนวนเต็มโดยใช้ฟังก์ชัน map() และคัดกรองค่าที่ไม่ใช่ '-' ออก
    numeric_values = list(map(int, [val for val in values if val != '-']))

    # แบ่งค่าเป็นกลุ่มของรายสัปดาห์
    weekly_values = [# numeric_values[:6]
                    numeric_values[i:i+7]
                     for i in range(0, len(numeric_values), 7)]
    

    # คำนวณค่าเฉลี่ยของแต่ละสัปดาห์
    weekly_averages = [np.mean(weekly) for weekly in weekly_values]
    return weekly_averages


# สร้างคอลัมน์ 'weekly_average' ใน DataFrame เพื่อเก็บค่าเฉลี่ยของรายสัปดาห์
grouped_df['weekly_average'] = grouped_df['value'].apply(
    calculate_weekly_average)

# แสดง DataFrame ที่ได้
grouped_df.drop(['value'], axis=1, inplace=True)
grouped_df .head()

,สถานี,Year,weekly_average
0,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,"[77.28571428571429, 78.28571428571429, 73.4285..."
1,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2014,"[82.0, 80.28571428571429, 77.57142857142857, 7..."
2,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2015,"[78.85714285714286, 79.28571428571429, 75.5714..."
3,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2016,"[76.42857142857143, 76.28571428571429, 72.5714..."
4,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2017,"[84.42857142857143, 81.57142857142857, 77.1428..."


In [8]:
# แยกข้อมูลออกจากกันในคอลัมน์ 'weekly_average' และสร้าง DataFrame ใหม่เพื่อนำมาเก็บข้อมูลใหม่ที่ทำการเพิ่มลำดับของสัปดาห์เริ่มต้น
new_data = []

# วนลูปเพื่อแบ่งข้อมูลในคอลัมน์ 'weekly_average' และสร้างข้อมูลใหม่
for index, row in grouped_df.iterrows():
    station = row['สถานี']
    year = row['Year']
    weekly_averages = row['weekly_average']

    # วนลูปเพื่อสร้างข้อมูลใหม่สำหรับแต่ละค่าในคอลัมน์ 'weekly_average'
    for i, avg in enumerate(weekly_averages):
        # คำนวณหาค่าของสัปดาห์เริ่มต้น
        start_of_week = i + 1
        new_data.append([station, year, start_of_week, avg])

# สร้าง DataFrame จากข้อมูลใหม่
new_df = pd.DataFrame(
    new_data, columns=['สถานี', 'Year', 'start_of_week', 'Humi'])

# แสดง DataFrame ที่ได้
new_df.head()

,สถานี,Year,start_of_week,Humi
0,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,1,77.285714
1,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,2,78.285714
2,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,3,73.428571
3,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,4,78.000000
4,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,5,72.142857


In [9]:
new_df = pd.concat([week1, new_df], ignore_index=True)
new_df.to_csv(
    f'./dataset/humi/Humi2013-2022.csv', index=False, encoding='utf-8-sig')
new_df.head()

,สถานี,Year,start_of_week,Humi
0,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,0,79.600000
1,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,1,77.285714
2,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,2,78.285714
3,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,3,73.428571
4,300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน,2013,4,78.000000


# ใช้ไม่ได้

In [ ]:
# สร้างรายการเพื่อเก็บค่าเฉลี่ยในทุกๆ 7 วัน
averages = []

for i, row in df.iterrows():
    # เลือกข้อมูลในคอลัมน์ที่มีค่าตั้งแต่ตำแหน่งที่ 2 เป็นต้นไป
    month_data = row[2:]
    month_averages = []
    for j in range(0, len(month_data), 7):
        week_data = month_data[j:j+7]
        # คำนวณค่าเฉลี่ยของทุกๆ 7 วัน
        week_average = week_data[week_data != '-'].astype(float).mean()
        month_averages.append(week_average)
    averages.extend(month_averages)

# สร้าง DataFrame ใหม่โดยรวมข้อมูลเฉลี่ยของทุกๆ 7 วัน
new_df = pd.DataFrame({
    'Date': pd.date_range(start=df['เดือน/ปี'].iloc[0], periods=len(averages), freq='7D'),
    'Average': averages
})

# พิมพ์ DataFrame ใหม่
print(new_df)

In [111]:
# แปลงข้อมูลในคอลัมน์ที่เลือกมาเป็นตัวเลข
df.iloc[:, 2:33] = df.iloc[:, 2:33].apply(pd.to_numeric, errors='coerce')

# สร้างคอลัมน์ใหม่สำหรับเก็บข้อมูลสรุปรายสัปดาห์
df['รายสัปดาห์ที่ 1'] = df.iloc[:, 2:9].mean(axis=1)  # col 2:9
df['รายสัปดาห์ที่ 2'] = df.iloc[:, 9:16].mean(axis=1)  # col 9:16
df['รายสัปดาห์ที่ 3'] = df.iloc[:, 16:23].mean(axis=1)  # col 16:23
df['รายสัปดาห์ที่ 4'] = df.iloc[:, 23:30].mean(axis=1)  # col 23:30

# main is columns
main = df[['สถานี', 'เดือน/ปี', 'รายสัปดาห์ที่ 1',
           'รายสัปดาห์ที่ 2', 'รายสัปดาห์ที่ 3', 'รายสัปดาห์ที่ 4']]

# แสดงผลลัพธ์
print(main)

# เพิ่มข้อมูลของเดือนถัดไปเพื่อให้สัปดาห์ที่ 4 นับต่อไปในเดือนถัดไป
next_month_data = {
    'สถานี': df['สถานี'],
    'เดือน/ปี': df['เดือน/ปี'].apply(pd.Period).apply(lambda x: x + pd.DateOffset(months=1)),
}
# เพิ่มคอลัมน์วันที่สำหรับสัปดาห์ที่ 4
next_month_data.update({f'วันที่ {i}': None for i in range(1, 8)})
next_month_df = pd.DataFrame(next_month_data)

# เพิ่มข้อมูลของเดือนถัดไปเข้า DataFrame
df = pd.concat([df, next_month_df], ignore_index=True)

# คำนวณค่าเฉลี่ยสำหรับสัปดาห์ที่ 4 ของเดือนถัดไป
# คอลัมน์ 30: เป็นคอลัมน์ของเดือนถัดไป
next_month_week_4 = df.iloc[:, 30:].mean(axis=1)
main['รายสัปดาห์ที่ 4 (เดือนถัดไป)'] = next_month_week_4

# แสดงผลลัพธ์
print(main)

                                สถานี   เดือน/ปี รายสัปดาห์ที่ 1  \
0      300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน 2013-01-01       79.428571   
1      300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน 2013-02-01       74.285714   
2      300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน 2013-03-01            67.0   
3      300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน 2013-04-01       50.285714   
4      300201-แม่ฮ่องสอน จ.แม่ฮ่องสอน 2013-05-01       73.571429   
...                               ...        ...             ...   
14958      583201-นราธิวาส จ.นราธิวาส 2022-08-01       81.428571   
14959      583201-นราธิวาส จ.นราธิวาส 2022-09-01            80.0   
14960      583201-นราธิวาส จ.นราธิวาส 2022-10-01       79.285714   
14961      583201-นราธิวาส จ.นราธิวาส 2022-11-01       86.142857   
14962      583201-นราธิวาส จ.นราธิวาส 2022-12-01       89.428571   

      รายสัปดาห์ที่ 2 รายสัปดาห์ที่ 3 รายสัปดาห์ที่ 4  
0           76.857143            78.0       71.714286  
1           72.142857       69.428571       64.857143  
2           58.

ValueError: Must supply freq for datetime value

In [134]:
df.iloc[:, 2:]

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,82.0,80.0,82.0,78.0,76.0,78.0,80.0,78.0,77.0,77.0,...,76.0,73.0,72.0,70.0,70.0,70.0,71.0,75.0,87.0,84.0
1,77.0,82.0,77.0,72.0,72.0,70.0,70.0,72.0,72.0,74.0,...,66.0,68.0,69.0,66.0,66.0,60.0,59.0,NaN,NaN,NaN
2,61.0,66.0,64.0,73.0,73.0,67.0,65.0,69.0,57.0,54.0,...,53.0,52.0,54.0,56.0,54.0,52.0,51.0,54.0,56.0,60.0
3,57.0,53.0,46.0,47.0,51.0,49.0,49.0,52.0,52.0,51.0,...,70.0,64.0,58.0,49.0,50.0,49.0,48.0,61.0,68.0,NaN
4,64.0,66.0,76.0,78.0,74.0,75.0,82.0,72.0,68.0,72.0,...,67.0,71.0,78.0,79.0,73.0,66.0,66.0,63.0,64.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,80.0,81.0,87.0,80.0,79.0,82.0,81.0,81.0,80.0,77.0,...,71.0,79.0,73.0,78.0,78.0,79.0,77.0,78.0,79.0,76.0
14959,76.0,79.0,77.0,82.0,85.0,78.0,83.0,81.0,77.0,78.0,...,70.0,71.0,74.0,79.0,81.0,77.0,73.0,77.0,81.0,NaN
14960,67.0,71.0,83.0,86.0,80.0,83.0,85.0,80.0,88.0,83.0,...,83.0,85.0,82.0,81.0,73.0,86.0,89.0,83.0,81.0,83.0
14961,85.0,83.0,84.0,90.0,90.0,89.0,82.0,78.0,78.0,90.0,...,74.0,80.0,83.0,83.0,81.0,82.0,80.0,82.0,80.0,NaN


In [142]:
import pandas as pd

# อ่านข้อมูล
df = df_main.copy()

# แปลงข้อมูลในคอลัมน์ที่เลือกมาเป็นตัวเลข
df.iloc[:, 2:] = df.iloc[:, 2:].apply(pd.to_numeric, errors='coerce')

# สร้าง DataFrame ใหม่สำหรับเก็บข้อมูลที่แปลงแล้ว
result_df = pd.DataFrame(columns=df.columns)

# วนลูปตามแต่ละบรรทัด
for index, row in df.iterrows():
    station_data = []  # เก็บข้อมูลสำหรับแต่ละสถานี
    day_count = 0  # จำนวนวันที่เก็บข้อมูลในสัปดาห์ปัจจุบัน

    # วนลูปเก็บข้อมูลของแต่ละคอลัมน์
    for col in df.columns[2:]:
        if day_count < 7:
            if pd.notna(row[col]):  # ตรวจสอบว่าข้อมูลไม่ใช่ค่าว่าง
                station_data.append(row[col])
                day_count += 1
            else:
                day_count += 1
                pass
        else:
            # คำนวณค่าเฉลี่ยของข้อมูลในสัปดาห์ปัจจุบัน
            week_mean = sum(station_data) / len(station_data)
            print(sum(station_data) , len(station_data))
            # เพิ่มข้อมูลค่าเฉลี่ยของสัปดาห์ลงใน DataFrame ผลลัพธ์
            result_df = pd.concat([result_df, pd.DataFrame(
                [row['สถานี'], row['เดือน/ปี']] + station_data + [week_mean]).T], ignore_index=True)
            # รีเซ็ตตัวแปร
            station_data = [row[col]]
            day_count = 1

    # หากมีข้อมูลค้างอยู่หลังจบบรรทัดสุดท้าย
    if station_data:
        week_mean = sum(station_data) / len(station_data)
        result_df = pd.concat([result_df, pd.DataFrame(
            [row['สถานี'], row['เดือน/ปี']] + station_data + [week_mean]).T], ignore_index=True)

# แสดงผลลัพธ์
print(result_df)
# result_df.to_csv('result.csv', index=False)
result_df.to_csv(f'dataset/humi/testHumi.csv', index=False, encoding='utf-8-sig')

KeyboardInterrupt: 